In [ ]:
using LinearAlgebraicRepresentation
import Pkg
Pkg.add("BenchmarkTools")
using BenchmarkTools

In [2]:
function frag_face_channel(in_chan, out_chan, V, EV, FE, sp_idx)
    run_loop = true
    while run_loop
        sigma = take!(in_chan)
        if sigma != -1
            put!(out_chan, frag_face(V, EV, FE, sp_idx, sigma))
        else
            run_loop = false
        end
    end
end



frag_face_channel (generic function with 1 method)

In [6]:
using BenchmarkTools
using SparseArrays
using LinearAlgebraicRepresentation

Lar=LinearAlgebraicRepresentation
 
V = Float64[
        0 0 0; 0 1 0;
        1 1 0; 1 0 0;
        0 0 1; 0 1 1;
        1 1 1; 1 0 1
    ]

    EV = sparse(Int8[
        -1  1  0  0  0  0  0  0;
        0 -1  1  0  0  0  0  0;
        0  0 -1  1  0  0  0  0;
        -1  0  0  1  0  0  0  0;
        -1  0  0  0  1  0  0  0;
        0 -1  0  0  0  1  0  0;
        0  0 -1  0  0  0  1  0;
        0  0  0 -1  0  0  0  1;
        0  0  0  0 -1  1  0  0;
        0  0  0  0  0 -1  1  0;
        0  0  0  0  0  0 -1  1;
        0  0  0  0 -1  0  0  1;
    ])

    FE = sparse(Int8[
        1  1  1 -1  0  0  0  0  0  0  0  0;
        0  0  0  0  0  0  0  0 -1 -1 -1  1;
        -1  0  0  0  1 -1  0  0  1  0  0  0;
        0 -1  0  0  0  1 -1  0  0  1  0  0;
        0  0 -1  0  0  0  1 -1  0  0  1  0;
        0  0  0  1 -1  0  0  1  0  0  0 -1;
    ])

@time frag_face_channel([2,3,4],[0,0,0],Lar.Points(V),EV,FE,[0,2])


LoadError: MethodError: no method matching take!(::Array{Int64,1})
Closest candidates are:
  take!(!Matched::Distributed.WorkerPool) at D:\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.5\Distributed\src\workerpool.jl:136
  take!(!Matched::IOStream) at iostream.jl:419
  take!(!Matched::Base.GenericIOBuffer{Array{UInt8,1}}) at iobuffer.jl:385
  ...